In [14]:
import os 
import subprocess
from transcriptomics import * 

# Drosophila Melanogaster

Download Genome

In [23]:
# Make directory 
output_dir = "raw_data/dmel/genome"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


# Download the genome
rsync_path = "rsync://hgdownload.soe.ucsc.edu/goldenPath/dm6/bigZips/dm6.fa.gz"
output_path = os.path.join(output_dir, "dm6.fa.gz")
command = f"rsync --partial --progress {rsync_path} {output_path}"
subprocess.run(command, shell=True)

dm6.fa.gz
    45153922 100%  531.63MB/s    0:00:00 (xfer#1, to-check=0/1)

sent 47134 bytes  received 27010 bytes  49429.33 bytes/sec
total size is 45153922  speedup is 609.00


CompletedProcess(args='rsync --partial --progress rsync://hgdownload.soe.ucsc.edu/goldenPath/dm6/bigZips/dm6.fa.gz raw_data/dmel/genome/dm6.fa.gz', returncode=0)

Unzip Genome

In [24]:
# Unzip the genome
output_path = "raw_data/dmel/genome/dm6.fa.gz"
command = f"gunzip {output_path}"
subprocess.run(command, shell=True)

CompletedProcess(args='gunzip raw_data/dmel/genome/dm6.fa.gz', returncode=0)

Unhide Genome

In [ ]:
# Unhide genome file 
output_path = "raw_data/dmel/genome/dm6.fa"
command = f"chflags nohidden {output_path} {output_path}" 
subprocess.run(command, shell=True)

CompletedProcess(args='mv raw_data/dmel/genome/dm6.fa raw_data/dmel/genome/dm6.fa', returncode=0)

Download Transcriptome

In [17]:
output_dir = "raw_data/dmel/transcriptome"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Download the transcriptome 
rsync_path = "rsync://hgdownload.soe.ucsc.edu/goldenPath/dm6/bigZips/genes/dm6.ncbiRefSeq.gtf.gz"
output_path = os.path.join(output_dir, "dm6.ncbiRefSeq.gtf.gz")
command = f"rsync --partial --progress {rsync_path} {output_path}"
subprocess.run(command, shell=True)

dm6.ncbiRefSeq.gtf.gz
     5133989 100%    6.87MB/s    0:00:00 (xfer#1, to-check=0/1)

sent 38 bytes  received 5135347 bytes  3423590.00 bytes/sec
total size is 5133989  speedup is 1.00


CompletedProcess(args='rsync --partial --progress rsync://hgdownload.soe.ucsc.edu/goldenPath/dm6/bigZips/genes/dm6.ncbiRefSeq.gtf.gz raw_data/dmel/transcriptome/dm6.ncbiRefSeq.gtf.gz', returncode=0)

Unzip Transcriptome

In [18]:
# Unzip the transcriptome 
output_path = "raw_data/dmel/transcriptome/dm6.ncbiRefSeq.gtf.gz"
command = f"gunzip {output_path}"
subprocess.run(command, shell=True)

gunzip: raw_data/dmel/transcriptome/dm6.ncbiRefSeq.gtf already exists -- skipping


CompletedProcess(args='gunzip raw_data/dmel/transcriptome/dm6.ncbiRefSeq.gtf.gz', returncode=1)

Unhide Transcriptome

In [30]:
# Unhide transcriptome file
output_path = "raw_data/dmel/transcriptome/dm6.ncbiRefSeq.gtf"
command = f"chflags nohidden {output_path}"
subprocess.run(command, shell=True)

CompletedProcess(args='chflags nohidden raw_data/dmel/transcriptome/dm6.ncbiRefSeq.gtf', returncode=0)

Build transcriptome object 

In [19]:
mel_genome_path = "raw_data/dmel/genome/dm6.fa"
mel_transcriptome_path = "raw_data/dmel/transcriptome/dm6.ncbiRefSeq.gtf"
mel_object_name = "mel_transcriptome"
update_transcriptome_object(mel_genome_path, mel_transcriptome_path, mel_object_name)

Found 17868 unique genes.


100%|██████████| 17868/17868 [00:24<00:00, 724.53it/s] 


Transcriptome(genes=17868)
Transcriptome object has been updated and saved to mel_transcriptome.pkl


In [20]:
tr = load_transcriptome_object(mel_object_name)

In [21]:
check_exons_contain_all_features(tr)

Export mRNA to Fasta file

In [35]:
output_dir = "raw_data/dmel/transcriptome/mRNA_no_introns"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_path = os.path.join(output_dir, "mRNA_no_introns.fasta")

with open(output_path, "w") as output_file:
    for gene_name in tr.genes.keys():
        gene = tr.genes[gene_name]
        for transcript in gene.transcripts:
            output_file.write(f">{transcript.name} gene={gene.name} location={transcript.chromosome}:{transcript.position[0]}-{transcript.position[1]} strand={transcript.strand} \n{transcript.mrna_sequence} \n")


Export mRNA (introns included) to Fasta file

In [36]:
output_dir = "raw_data/dmel/transcriptome/mRNA_yes_introns"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_path = os.path.join(output_dir, "mRNA_yes_introns.fasta")

with open(output_path, "w") as output_file:
    for gene_name in tr.genes.keys():
        gene = tr.genes[gene_name]
        for transcript in gene.transcripts:
            output_file.write(f">{transcript.name} gene={gene.name} location={transcript.chromosome}:{transcript.position[0]}-{transcript.position[1]} strand={transcript.strand} \n{transcript.dna_sequence} \n")


Create Blast Databases

In [37]:
# Check that makeblastdb is installed
!makeblastdb -version

makeblastdb: 2.15.0+
 Package: blast 2.15.0, build Oct 19 2023 15:16:13


In [38]:
input_path = "raw_data/dmel/transcriptome/mRNA_no_introns/mRNA_no_introns.fasta"
output_path = "raw_data/dmel/transcriptome/mRNA_no_introns/mRNA_no_introns"
command = f"makeblastdb -in {input_path} -dbtype nucl -parse_seqids -out {output_path}"
subprocess.run(command, shell=True)



Building a new DB, current time: 11/11/2024 12:12:02
New DB name:   /Users/giacomo.glotzer/Desktop/Drosophila-HCR/raw_data/dmel/transcriptome/mRNA_no_introns/mRNA_no_introns
New DB title:  raw_data/dmel/transcriptome/mRNA_no_introns/mRNA_no_introns.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /Users/giacomo.glotzer/Desktop/Drosophila-HCR/raw_data/dmel/transcriptome/mRNA_no_introns/mRNA_no_introns
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 35121 sequences in 0.849447 seconds.




CompletedProcess(args='makeblastdb -in raw_data/dmel/transcriptome/mRNA_no_introns/mRNA_no_introns.fasta -dbtype nucl -parse_seqids -out raw_data/dmel/transcriptome/mRNA_no_introns/mRNA_no_introns', returncode=0)

In [39]:
input_path = "raw_data/dmel/transcriptome/mRNA_yes_introns/mRNA_yes_introns.fasta"
output_path = "raw_data/dmel/transcriptome/mRNA_yes_introns/mRNA_yes_introns"
command = f"makeblastdb -in {input_path} -dbtype nucl -parse_seqids -out {output_path}"
subprocess.run(command, shell=True)



Building a new DB, current time: 11/11/2024 12:12:03
New DB name:   /Users/giacomo.glotzer/Desktop/Drosophila-HCR/raw_data/dmel/transcriptome/mRNA_yes_introns/mRNA_yes_introns
New DB title:  raw_data/dmel/transcriptome/mRNA_yes_introns/mRNA_yes_introns.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /Users/giacomo.glotzer/Desktop/Drosophila-HCR/raw_data/dmel/transcriptome/mRNA_yes_introns/mRNA_yes_introns
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 35121 sequences in 1.77955 seconds.




CompletedProcess(args='makeblastdb -in raw_data/dmel/transcriptome/mRNA_yes_introns/mRNA_yes_introns.fasta -dbtype nucl -parse_seqids -out raw_data/dmel/transcriptome/mRNA_yes_introns/mRNA_yes_introns', returncode=0)

# Drosophila Yakuba
...